### Imports

In [1]:
import sys
sys.path.append("llmapslab")
import os
import json
import re
from  openai import OpenAI
from langchain_openai import ChatOpenAI

### setup openai

#### Open AI api access setup 
- Api access account is not same as the chatgpt account. 
- setup your project and api key from here https://platform.openai.com/playground/chat?models=gpt-4o-mini-2024-07-18
- create a new api key and copy and save it before closing the pop up window. Once the window is closed 
 the key is no more accessable.
- before making call to openai ensure you have  balance and keep track of your cost
- come back to the playground and try in the gui a simple chat to ensure chat is working
https://platform.openai.com/organization/usage

In [2]:
with open('secrets.json', 'r') as jsonfile:
    configs = json.load(jsonfile)

client = OpenAI(api_key=configs.get('openai_api_key'),
                # organization='Personal',
                # project='proj_DqdKUU38qOGVj9Qxdq6UlXcD',
               )

In [3]:
def call_openai_api(client, messages, model="gpt-3.5-turbo", max_retries=5, wait_time=5):
    retries = 0
    while retries < max_retries:
        try:
            # Make a request to the OpenAI ChatCompletion API
           response = client.chat.completions.create(
              model="gpt-4o-mini",
              messages=messages,
           )
           return response
        except client.error.RateLimitError:
            print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            retries += 1
    raise Exception("Max retries exceeded. Please check your plan and billing details.")

# Define the conversation messages
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]

# Call the API
response = call_openai_api(client, messages)

In [4]:
print(response)

ChatCompletion(id='chatcmpl-9onaGO3TrI5Ni42oMMgj3xdt7YP7L', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is Paris.', role='assistant', function_call=None, tool_calls=None))], created=1721894028, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_661538dc1f', usage=CompletionUsage(completion_tokens=7, prompt_tokens=24, total_tokens=31))


In [5]:
response.choices[0].message

ChatCompletionMessage(content='The capital of France is Paris.', role='assistant', function_call=None, tool_calls=None)

In [6]:
response.choices[0].message.content

'The capital of France is Paris.'

### langchain ChatOpenAI call 

In [7]:
openai_api_key = configs.get('openai_api_key')
messages = [
    (
        "system",
        "You are a helpful translator. Translate the user sentence to French.",
    ),
    ("human", "I love programming."),
]

llm = ChatOpenAI(
    api_key=openai_api_key,
    model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o"
    )
response = llm.invoke(messages)
print(response)

content="J'adore la programmation." response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 28, 'total_tokens': 33}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_611b667b19', 'finish_reason': 'stop', 'logprobs': None} id='run-8fc965ae-6800-4111-bb65-1d97ba404354-0' usage_metadata={'input_tokens': 28, 'output_tokens': 5, 'total_tokens': 33}


In [8]:
response.content

"J'adore la programmation."

In [9]:
messages = [
    (
        "system",
        "You are an assistant",
    ),
    ("human", "generate a code to read excel"),
]

llm = ChatOpenAI(
    api_key=openai_api_key,
    model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o"
    )
response = llm.invoke(messages)
print(response)

content='To read Excel files in Python, you can use the `pandas` library, which is a powerful data manipulation and analysis library. The `pandas` library has built-in support for reading Excel files through the `read_excel` function.\n\nHere\'s a step-by-step guide to reading an Excel file using `pandas`:\n\n### Step 1: Install the Required Libraries\n\nMake sure you have `pandas` and `openpyxl` installed. You can install them using pip if you haven\'t done so:\n\n```bash\npip install pandas openpyxl\n```\n\n### Step 2: Read an Excel File\n\nHere\'s a sample code snippet to read an Excel file:\n\n```python\nimport pandas as pd\n\n# Specify the path to your Excel file\nfile_path = \'path/to/your/excel_file.xlsx\'\n\n# Read the Excel file\n# If you want to read a specific sheet, you can specify the sheet name or index\ndf = pd.read_excel(file_path, sheet_name=\'Sheet1\')  # or use sheet_name=0 for the first sheet\n\n# Display the DataFrame\nprint(df)\n\n# Optionally, you can also perfor

In [10]:
print(response.content)

To read Excel files in Python, you can use the `pandas` library, which is a powerful data manipulation and analysis library. The `pandas` library has built-in support for reading Excel files through the `read_excel` function.

Here's a step-by-step guide to reading an Excel file using `pandas`:

### Step 1: Install the Required Libraries

Make sure you have `pandas` and `openpyxl` installed. You can install them using pip if you haven't done so:

```bash
pip install pandas openpyxl
```

### Step 2: Read an Excel File

Here's a sample code snippet to read an Excel file:

```python
import pandas as pd

# Specify the path to your Excel file
file_path = 'path/to/your/excel_file.xlsx'

# Read the Excel file
# If you want to read a specific sheet, you can specify the sheet name or index
df = pd.read_excel(file_path, sheet_name='Sheet1')  # or use sheet_name=0 for the first sheet

# Display the DataFrame
print(df)

# Optionally, you can also perform further operations on the DataFrame
# For e

### Handcrafted Agent 

#### what is agent doing
- calling model with a prompt
- maintianing a history of conversation

In [11]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            temperature=0,
            messages=self.messages)
        return completion.choices[0].message.content

### A ReAct type prompt for agent 

- Thought:
  - user asks a question to model
  - model think about a plan of action from the available actions
   - Available Actions:
     - name of the python function: parameters of the function
     - identifying the parameters correctly is critical
- Action:
  - from the free flow text identify the parameters for  the action
  - return the model output as ACtion: function name: paramer
  - This output will be parsed by the user or another program later to call the function
  - also output 'PAUSE'. Why is this required ? may be external program can use this somehow.
- Observation:
  - assume that the function was called and result was obrained by user or by anothe program
  - the model is now again called with the result along with the history
  - the model now get to see:
     - The original quesiton from the user
     - The actions that it had suggested in terms of function name and its parameters
     - the  result from the action
- Answer: provide the final answer from the observation and the history of messages as context
- Demonstration: give a task demonstration

In [12]:
agent_react_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

### python functions for the agent

In [13]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

### Run the agent to get the action name

In [14]:
abot = Agent(agent_react_prompt)

In [15]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I need to find the average weight of a Toy Poodle using the average_dog_weight action. 
Action: average_dog_weight: Toy Poodle
PAUSE


### Parse the action name and parameters and manually recall the model for answer

In [16]:
def parse_action_name_n_params(result):
    result.split('\n')
    pattern_action = re.compile('^Action: (\w+): (.*)$')
    parsed_actions = []
    for line in result.split('\n'):
        action_found = pattern_action.match(line)
        if action_found:
            parsed_actions.append(action_found)
    print(parsed_actions)
    if parsed_actions:
        action, action_input = parsed_actions[0].groups()
        print(action, action_input )
        return action, action_input
    return
parse_action_name_n_params(result)

[<re.Match object; span=(0, 38), match='Action: average_dog_weight: Toy Poodle'>]
average_dog_weight Toy Poodle


('average_dog_weight', 'Toy Poodle')

In [17]:
result = average_dog_weight("Toy Poodle")
print(result)
next_prompt = "Observation: {}".format(result)
abot(next_prompt)

a toy poodles average weight is 7 lbs


'Answer: A Toy Poodle weighs 7 lbs.'

### Check agent is able to identify different action based on users question

In [18]:
result = abot("what is the result of multiplication of 5 and 20?")
print(result)

Thought: I need to perform a multiplication calculation to find the result of 5 multiplied by 20. 
Action: calculate: 5 * 20
PAUSE


### Automate the Thought to Answer by calling the agent multiple times till the Final Answer

In [23]:
def autoagent(question, max_turns=5):
    i = 0
    agent = Agent(agent_react_prompt)
    user_prompt = question
    while i < max_turns:
        i += 1
        result = agent(user_prompt)
        print(result)
        parse_result = parse_action_name_n_params(result)
        if isinstance( parse_result, tuple):
            action, action_input = parse_result
            if action not in known_actions:
                raise Exception("Unknown actions: {}: {}".format(action, action_input))
            print(f'calling {action}: with arguments: {action_input}')            
            observation = known_actions[action](action_input)
            print('observation:', observation)
            user_prompt = "Observation: {}".format(observation)
        else:
            return    

In [24]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
autoagent(question)

Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. I'll start by looking up the weight of each breed. 

Action: average_dog_weight: Border Collie
PAUSE
[<re.Match object; span=(0, 41), match='Action: average_dog_weight: Border Collie'>]
average_dog_weight Border Collie
calling average_dog_weight: with arguments: Border Collie
observation: a Border Collies average weight is 37 lbs
Thought: Now that I have the weight of the Border Collie, I need to find the average weight of the Scottish Terrier to complete the calculation for their combined weight.

Action: average_dog_weight: Scottish Terrier
PAUSE
[<re.Match object; span=(0, 44), match='Action: average_dog_weight: Scottish Terrier'>]
average_dog_weight Scottish Terrier
calling average_dog_weight: with arguments: Scottish Terrier
observation: Scottish Terriers average 20 lbs
Thought: I now have the average weights for both dogs: the Border Collie weighs 37 lbs 